# Node Classification
This tutorial shows how to train a multi-layer GraphSAGE for node
classification on ``ogbn-arxiv`` provided by [Open Graph
Benchmark (OGB)](https://ogb.stanford.edu/). The dataset contains around
170 thousand nodes and 1 million edges.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dmlc/dgl/blob/master/notebooks/stochastic_training/node_classification.ipynb) [![GitHub](https://img.shields.io/badge/-View%20on%20GitHub-181717?logo=github&logoColor=ffffff)](https://github.com/dmlc/dgl/blob/master/notebooks/stochastic_training/node_classification.ipynb)

By the end of this tutorial, you will be able to

-  Train a GNN model for node classification on a single GPU with DGL's
   neighbor sampling components.

## Install DGL package

In [1]:
# Install required packages.
import os
import torch
import numpy as np
os.environ['TORCH'] = torch.__version__
os.environ['DGLBACKEND'] = "pytorch"

# Install the CPU version. If you want to install CUDA version, please
# refer to https://www.dgl.ai/pages/start.html.
device = torch.device("cpu")
!pip install --pre dgl -f https://data.dgl.ai/wheels-test/repo.html

try:
    import dgl
    import dgl.graphbolt as gb
    installed = True
except ImportError as error:
    installed = False
    print(error)
print("DGL installed!" if installed else "DGL not found!")

Looking in links: https://data.dgl.ai/wheels-test/repo.html


DGL installed!


## Loading Dataset
`ogbn-arxiv` is already prepared as ``BuiltinDataset`` in **GraphBolt**.

In [2]:
dataset = gb.BuiltinDataset("ogbn-arxiv").load()

datasets/ogbn-arxiv.zip:   0%|                                       | 0.00/88.1M [00:00<?, ?B/s]

datasets/ogbn-arxiv.zip:   8%|██▎                           | 6.61M/88.1M [00:00<00:01, 49.3MB/s]

datasets/ogbn-arxiv.zip:  13%|███▉                          | 11.5M/88.1M [00:00<00:02, 34.3MB/s]

datasets/ogbn-arxiv.zip:  19%|█████▋                        | 16.8M/88.1M [00:00<00:02, 31.2MB/s]

datasets/ogbn-arxiv.zip:  27%|███████▉                      | 23.4M/88.1M [00:00<00:02, 25.9MB/s]

...

datasets/ogbn-arxiv.zip: 100%|█████████████████████████████▉| 88.0M/88.1M [00:03<00:00, 18.9MB/s]

datasets/ogbn-arxiv.zip: 100%|██████████████████████████████| 88.1M/88.1M [00:03<00:00, 28.3MB/s]

Extracting file to datasets


The dataset is already preprocessed.


/home/ubuntu/regression_test/dgl/python/dgl/graphbolt/impl/ondisk_dataset.py:467: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(graph_topology.path)


Dataset consists of graph, feature and tasks. You can get the training-validation-test set from the tasks. Seed nodes and corresponding labels are already stored in each training-validation-test set. Other metadata such as number of classes are also stored in the tasks. In this dataset, there is only one task: `node classification`.

In [3]:
graph = dataset.graph
feature = dataset.feature
train_set = dataset.tasks[0].train_set
valid_set = dataset.tasks[0].validation_set
test_set = dataset.tasks[0].test_set
task_name = dataset.tasks[0].metadata["name"]
num_classes = dataset.tasks[0].metadata["num_classes"]
print(f"Task: {task_name}. Number of classes: {num_classes}")

Task: node_classification. Number of classes: 40


## How DGL Handles Computation Dependency¶
The computation dependency for message passing of a single node can be described as a series of message flow graphs (MFG).

![DGL Computation](https://data.dgl.ai/tutorial/img/bipartite.gif)

## Defining Neighbor Sampler and Data Loader in DGL

DGL provides tools to iterate over the dataset in minibatches while generating the computation dependencies to compute their outputs with the MFGs above. For node classification, you can use `dgl.graphbolt.DataLoader` for iterating over the dataset. It accepts a data pipe that generates minibatches of nodes and their labels, sample neighbors for each node, and generate the computation dependencies in the form of MFGs. Feature fetching, block creation and copying to target device are also supported. All these operations are split into separate stages in the data pipe, so that you can customize the data pipeline by inserting your own operations.

Let’s say that each node will gather messages from 4 neighbors on each layer. The code defining the data loader and neighbor sampler will look like the following.


In [4]:
datapipe = gb.ItemSampler(train_set, batch_size=1024, shuffle=True)
datapipe = datapipe.sample_neighbor(graph, [4, 4])
datapipe = datapipe.fetch_feature(feature, node_feature_keys=["feat"])
datapipe = datapipe.copy_to(device)
train_dataloader = gb.DataLoader(datapipe, num_workers=0)

You can iterate over the data loader and a `MiniBatch` object is yielded.



In [5]:
data = next(iter(train_dataloader))
print(data)

AssertionError: Data type of nodes must be consistent with indices.dtype(torch.int64), but got torch.int32.
This exception is thrown by __iter__ of NeighborSampler(datapipe=ShardingFilterIterDataPipe, deduplicate=True, fanouts=[tensor([4]), tensor([4])], graph=FusedCSCSamplingGraph(csc_indptr=tensor([      0,     289,     290,  ..., 1166240, 1166243, 1166243]),
                      indices=tensor([ 75652, 105878,  59944,  ...,  35711, 103121,  30351]),
                      num_nodes=169343, num_edges=1166243, node_attributes={}, edge_attributes={}), prob_name=None, replace=False)

You can get the input node IDs from MFGs.

In [6]:
mfgs = data.blocks
input_nodes = mfgs[0].srcdata[dgl.NID]
print(f"Input nodes: {input_nodes}.")

NameError: name 'data' is not defined

## Defining Model
Let’s consider training a 2-layer GraphSAGE with neighbor sampling. The model can be written as follows:



In [7]:
import torch.nn as nn
import torch.nn.functional as F
from dgl.nn import SAGEConv


class Model(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(Model, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, aggregator_type="mean")
        self.conv2 = SAGEConv(h_feats, num_classes, aggregator_type="mean")
        self.h_feats = h_feats

    def forward(self, mfgs, x):
        h = self.conv1(mfgs[0], x)
        h = F.relu(h)
        h = self.conv2(mfgs[1], h)
        return h


in_size = feature.size("node", None, "feat")[0]
model = Model(in_size, 64, num_classes).to(device)

## Defining Training Loop

The following initializes the model and defines the optimizer.


In [8]:
opt = torch.optim.Adam(model.parameters())

When computing the validation score for model selection, usually you can also do neighbor sampling. To do that, you need to define another data loader.

In [9]:
datapipe = gb.ItemSampler(valid_set, batch_size=1024, shuffle=False)
datapipe = datapipe.sample_neighbor(graph, [4, 4])
datapipe = datapipe.fetch_feature(feature, node_feature_keys=["feat"])
datapipe = datapipe.copy_to(device)
valid_dataloader = gb.DataLoader(datapipe, num_workers=0)


import sklearn.metrics

The following is a training loop that performs validation every epoch. It also saves the model with the best validation accuracy into a file.

In [10]:
import tqdm

for epoch in range(10):
    model.train()

    with tqdm.tqdm(train_dataloader) as tq:
        for step, data in enumerate(tq):
            x = data.node_features["feat"]
            labels = data.labels

            predictions = model(data.blocks, x)

            loss = F.cross_entropy(predictions, labels)
            opt.zero_grad()
            loss.backward()
            opt.step()

            accuracy = sklearn.metrics.accuracy_score(
                labels.cpu().numpy(),
                predictions.argmax(1).detach().cpu().numpy(),
            )

            tq.set_postfix(
                {"loss": "%.03f" % loss.item(), "acc": "%.03f" % accuracy},
                refresh=False,
            )

    model.eval()

    predictions = []
    labels = []
    with tqdm.tqdm(valid_dataloader) as tq, torch.no_grad():
        for data in tq:
            x = data.node_features["feat"]
            labels.append(data.labels.cpu().numpy())
            predictions.append(model(data.blocks, x).argmax(1).cpu().numpy())
        predictions = np.concatenate(predictions)
        labels = np.concatenate(labels)
        accuracy = sklearn.metrics.accuracy_score(labels, predictions)
        print("Epoch {} Validation Accuracy {}".format(epoch, accuracy))

        # Note that this tutorial do not train the whole model to the end.
        break

0it [00:00, ?it/s]

0it [00:00, ?it/s]

AssertionError: Data type of nodes must be consistent with indices.dtype(torch.int64), but got torch.int32.
This exception is thrown by __iter__ of NeighborSampler(datapipe=ShardingFilterIterDataPipe, deduplicate=True, fanouts=[tensor([4]), tensor([4])], graph=FusedCSCSamplingGraph(csc_indptr=tensor([      0,     289,     290,  ..., 1166240, 1166243, 1166243]),
                      indices=tensor([ 75652, 105878,  59944,  ...,  35711, 103121,  30351]),
                      num_nodes=169343, num_edges=1166243, node_attributes={}, edge_attributes={}), prob_name=None, replace=False)

## Conclusion

In this tutorial, you have learned how to train a multi-layer GraphSAGE with neighbor sampling.
